In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from binerizer import *
from DNFRuleModel import DNFRuleModel
from MasterModel import MasterModel

In [15]:
int(500/8)

62

In [2]:
credit = pd.read_csv('data/credit.csv')
data = binerizeData(credit, verbose = False)
X = data.to_numpy()[:,0:164]
Y = data.to_numpy()[:,165]

#X[Y,0] = 1
#X[~Y,0] = 0

ruleMod = DNFRuleModel(X, Y)


In [3]:
np.ones(numFeatures)

NameError: name 'numFeatures' is not defined

In [3]:
numSamples, numFeatures = X.shape
model = gp.Model('pricing')
D = 10

z = []
for k in range(numFeatures):
    z.append(model.addVar(vtype=GRB.BINARY, name="z[%d]"%k))

delta = []
for i in range(numSamples):
    delta.append(model.addVar(vtype=GRB.BINARY, name= "delta[%d]"%i))

complexConst = model.addConstr(gp.LinExpr(np.ones(numFeatures), z) <= D, name="ComplexityConst")

for i in range(numSamples):
    if not Y[i]:
         model.addConstr(gp.LinExpr(~X[i,:]*1, z) + delta[i] >= 1, name="sampleConstraint[%d]"%i)
    else:
        for j in range(numFeatures):
            if not X[i,j]:
                model.addConstr(delta[i]+z[j] <= 1,name="sampleConstraint[%d]-%d"%(i,j))
                
model.write('pricing_test.lp')


Using license file /Users/connorlawless/gurobi.lic
Academic license - for non-commercial use only


In [9]:
dz = gp.LinExpr(np.ones(sum(~Y)), np.array(delta)[~Y])
dp = gp.LinExpr(np.ones(sum(Y)), np.array(delta)[Y])
lam = gp.LinExpr(1.5*np.ones(len(z)), z)
lam_constant = gp.LinExpr(1.5, 1)

dz.add(dp)
dz.add(lam)
dz.add(lam_constant)

model.optimize()

GurobiError: Second argument should be a list of Var